# Feature Focused Data Architecture

This workflow examines data architecture optimizations for making data more useful for ML features. That sounds opinionated and it probably is.  While there is no perfect way there are tips that will make MLOps process more manageable, scalable, and useful. 

Machine Learning (ML) is far more than just training a model:

1. Find data sources
    - discovery
    - understandinng
    - formats
    - frequency
    - preparation, ETL
2. Combine data sources
    - formats
    - frequency
    - preparation, ETL
3. Feature Enginneering
    - Converting raw data columns into useful signal for ML methods
4. Training ML Models
    - Splits for train/validate/test
    - Iterate Features and Feature Engineering
5. Evaluate Models
    - Continously
6. Serve Models
    - Format features for predition
    - Serve features for prediction
7. Monitor Models
    - Skew: Change from training
    - Drift: Change over time
    - Continously
    - Monitor Features for change
    
When the goal is training a model (4), it might seem easy to ad-hoc work through 1-3. _Let's be honest - it's what we do most of the time._ But then, when a model version proves useful, many compromises are needed to get 5-7 to ~~work~~ - it rarely works correctly.

**What if**
- you could make careful decision during 1-3 that could essentially automate 5-7 seemlessly?
- it was not hard or time consuming?
- it makes it easier to train and iterate?
- it made everything easier?


# How? BigQuery!
BigQuery is a data warehouse right?  That seems perfect for 1-3 ... until you have fast changing data and low latency serving needs.  Actually, its perfect for this as well.  Let's proceed and discover together!


---
## Colab Setup

When running this notebook in [Colab](https://colab.google/) or [Colab Enterprise](https://cloud.google.com/colab/docs/introduction), this section will authenticate to GCP (follow prompts in the popup) and set the current project for the session.

In [1]:
PROJECT_ID = 'statmike-mlops-349915' # replace with project ID

In [2]:
try:
    from google.colab import auth
    auth.authenticate_user()
    !gcloud config set project {PROJECT_ID}
    print('Colab authorized to GCP')
except Exception:
    print('Not a Colab Environment')
    pass

Not a Colab Environment


---
## Installs

The list `packages` contains tuples of package import names and install names.  If the import name is not found then the install name is used to install quitely for the current user.

In [6]:
# tuples of (import name, install name, min_version)
packages = [
    ('google.cloud.bigquery', 'google-cloud-bigquery'),
    ('google.cloud.storage', 'google-cloud-storage')
]

import importlib
install = False
for package in packages:
    if not importlib.util.find_spec(package[0]):
        print(f'installing package {package[1]}')
        install = True
        !pip install {package[1]} -U -q --user
    elif len(package) == 3:
        if importlib.metadata.version(package[0]) < package[2]:
            print(f'updating package {package[1]}')
            install = True
            !pip install {package[1]} -U -q --user

### Restart Kernel (If Installs Occured)

After a kernel restart the code submission can start with the next cell after this one.

In [7]:
if install:
    import IPython
    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

---
## Setup

Inputs

In [8]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'statmike-mlops-349915'

In [9]:
REGION = 'us-central1'
EXPERIMENT = 'architecture'
SERIES = 'feature-data-architecture'

# BigQuery Parameters
BQ_PROJECT = PROJECT_ID
BQ_DATASET = SERIES
BQ_TABLE = EXPERIMENT
BQ_REGION = REGION[0:2]

# specify a GCS Bucket
GCS_BUCKET = PROJECT_ID

Packages

In [10]:
from google.cloud import storage
from google.cloud import bigquery

Clients

In [14]:
# gcs client: assumes bucket already exists
gcs = storage.Client(project = PROJECT_ID)
bucket = gcs.bucket(GCS_BUCKET)

# bigquery client
bq = bigquery.Client(project = PROJECT_ID)
%load_ext google.cloud.bigquery

The google.cloud.bigquery extension is already loaded. To reload it, use:
  %reload_ext google.cloud.bigquery


---
## Experiment

In [17]:
%%bigquery
CREATE OR REPLACE TABLE dev.features_source AS
            SELECT 'customer_abc' AS entity_id, 'a string 1' AS feature_1, 123 AS feature_2, CURRENT_DATE() AS feature_3, 'some words' AS feature_4, 1 AS feature_5, DATE_SUB(CURRENT_DATE(), INTERVAL CAST(FLOOR(100*RAND()) AS INT64) DAY) as feature_6
  UNION ALL SELECT 'customer_abd' AS entity_id, 'a string 2' AS feature_1, 124 AS feature_2, CURRENT_DATE() AS feature_3, 'some words' AS feature_4, 2 AS feature_5, NULL as feature_6
  UNION ALL SELECT 'customer_abe' AS entity_id, 'a string 1' AS feature_1, 121 AS feature_2, CURRENT_DATE() AS feature_3, 'some words' AS feature_4, NULL AS feature_5, DATE_SUB(CURRENT_DATE(), INTERVAL CAST(FLOOR(100*RAND()) AS INT64) DAY) as feature_6
  UNION ALL SELECT 'customer_abf' AS entity_id, 'a string 2' AS feature_1, 120 AS feature_2, CURRENT_DATE() AS feature_3, NULL AS feature_4, 4 AS feature_5, DATE_SUB(CURRENT_DATE(), INTERVAL CAST(FLOOR(100*RAND()) AS INT64) DAY) as feature_6
;
SELECT * FROM dev.features_source

Query is running:   0%|          |

Downloading:   0%|          |

,entity_id,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6
0,customer_abe,a string 1,121,2024-03-10,some words,<NA>,2024-01-26
1,customer_abc,a string 1,123,2024-03-10,some words,1,2023-12-26
2,customer_abd,a string 2,124,2024-03-10,some words,2,NaT
3,customer_abf,a string 2,120,2024-03-10,None,4,2024-02-18
